In [1]:
import os

In [2]:
%pwd

'c:\\Users\\49179\\Desktop\\Food_image_classification\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\49179\\Desktop\\Food_image_classification'

In [15]:
%%writefile src\Food_Classification\entity\config_entity.py
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

Overwriting src\Food_Classification\entity\config_entity.py


In [16]:
%%writefile src\Food_Classification\config\configuration.py
from Food_Classification.constants import *
from Food_Classification.utils.common import read_yaml,create_directory
from Food_Classification.entity.config_entity import DataIngestionConfig

class ConfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)

        create_directory([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
            config = self.config.data_ingestion

            create_directory([config.root_dir])

            data_ingestion_config = DataIngestionConfig(
                root_dir= config.root_dir,
                source_URL= config.source_url,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir)
            
            return data_ingestion_config
        

Overwriting src\Food_Classification\config\configuration.py


In [19]:
%%writefile src\Food_Classification\components\data_ingestion.py
import os 
from pathlib import Path
import zipfile
from Food_Classification import logger
from Food_Classification.utils.common import get_size
from Food_Classification.entity.config_entity import DataIngestionConfig
import gdown


class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config=config
        
    def download_file(self):
        data_url = self.config.source_URL
        if not os.path.exists(self.config.local_data_file):
            file_id = data_url.split('/')[-2]
            prefix = "https://drive.google.com/uc?export=download&id="
            gdown.download(prefix + file_id, self.config.local_data_file)
            logger.info(f"Downloaded {self.config.local_data_file}")

        else:
            logger.info(f"{self.config.local_data_file} already exists of size : {get_size(Path(self.config.local_data_file))}")

    
    def unzip_file(self):
        """
        Unzips the downloaded file.
        Extracts all the files from the zip file into the unzip directory.
        """

        unzip_dir = self.config.unzip_dir
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_dir)

            


Writing src\Food_Classification\components\data_ingestion.py


In [20]:
%%writefile src\Food_Classification\pipeline\stage_1_data_ingestion.py
from Food_Classification.config.configuration import ConfigurationManager
from Food_Classification.components.data_ingestion import DataIngestion
from Food_Classification import logger

STAGE_NAME = "Data Ingestion stage"

class DataIngestionTrainingPipeline:
    def __init__(self):
        pass

    def main(self):
        try:
            config = ConfigurationManager()
            data_ingestion_config = config.get_data_ingestion_config()
            data_ingestion = DataIngestion(config = data_ingestion_config)
            data_ingestion.download_file()
            data_ingestion.unzip_file()
        except Exception as e:
            raise e
        

if __name__ == "__main__":
    try:
        logger.info(f">>>>>>> stage : {STAGE_NAME} <<<<<<<<")
        obj = DataIngestionTrainingPipeline()
        obj.main()
        logger.info(f">>>>>>> stage : {STAGE_NAME} completed <<<<<<<< \n\nx========x")

    except Exception as e:
        logger.exception(e)
        raise e

Writing src\Food_Classification\pipeline\stage_1_data_ingestion.py
